In [ ]:
from typing import NamedTuple
import tracer

# A very simple example

In [ ]:
class User(NamedTuple):
    metric1: tracer.Float
    metric2: tracer.Float
    metric3: tracer.Int


@tracer.trace
def make_simplest_decision(user: User) -> bool:
    return user.metric3 > 5

The function returns the same plain value as it always would without tracing:

In [5]:
make_simplest_decision(User(metric1=0.6, metric2=0.6, metric3=6))

True

But we can also ask for a log of what it computed:

In [6]:
print(make_simplest_decision.trace.pretty_print())

return (metric3 > 5)


Obviously, we can use more complicated expressions:

In [ ]:
@tracer.trace
def make_slightly_more_complicated_decision(user: User) -> bool:
    return user.metric1 > 0.3 and user.metric2 < 0.4 and user.metric3 > 5


make_slightly_more_complicated_decision(User(metric1=0.6, metric2=0.6, metric3=6))

False

In [11]:
print(make_slightly_more_complicated_decision.trace.pretty_print())

if (metric1 > 0.3000) (=True):
  if (metric2 < 0.4000) (=False):
    return (metric2 < 0.4000)


Notice that the compiler converted the sequence of `and` operations into nested if-statements. That's because like in C, Python's boolean operations use short-circuition.

# Functions and condition elision

In [ ]:
def _compute_weighted_score(user: User) -> float:
    return user.metric1 * 0.7 + user.metric2 * 0.3


def _is_safe_to_proceed(score: float) -> bool:
    if score < 0.2:
        return False
    return True


def _check_system_status() -> bool:
    # Condition that does not depend on User
    maintenance_mode = False
    if maintenance_mode:
        return False
    return True


@tracer.trace
def make_decision_hierarchically(user: User) -> bool:
    if not _check_system_status():
        return False

    weighted_score = _compute_weighted_score(user)

    if not _is_safe_to_proceed(weighted_score):
        return False

    my_threshold = 0.4 * 2
    if weighted_score > my_threshold:
        return True

    # Cascade another check dependent on original values
    if user.metric3 > 5:
        return True

    return False


make_decision_hierarchically(User(metric1=0.9, metric2=0.9, metric3=1))

True

In [15]:
print(make_decision_hierarchically.trace.pretty_print())

if (((metric1 * 0.7000) + (metric2 * 0.3000)) < 0.2000) (=False):
  if (((metric1 * 0.7000) + (metric2 * 0.3000)) > 0.8000) (=True):
    return True


There are two important things to notice:
1. All the subroutines have been inlined in the trace. This makes it easier to just follow the flow.
2. Conditional statements that don't depend on `Traceable` objects are elided from the log. None of the fuss of `_check_system_status` appears in the log.
3. Similarly, aritmetic operations that don't depend on Traceables have been folded into a constant. Notably, `my_threshold` doesn't appear anywhere in the trace because it doesn't depend on a `Traceable`.

# The example from the README

In [16]:
class Patient(NamedTuple):
    blood_pressure: tracer.Float
    heart_rate: tracer.Int
    temperature: tracer.Float


def calculate_risk_score(
    blood_pressure: tracer.Float, heart_rate: tracer.Int
) -> tracer.Float:
    """Calculate a risk score based on blood pressure and heart rate."""
    pressure_factor = blood_pressure / 100.0
    rate_factor = heart_rate / 80.0
    return pressure_factor * rate_factor


def is_critical_temperature(temperature: tracer.Float) -> bool:
    """Check if temperature indicates a critical condition."""
    return temperature > 37.5


@tracer.trace
def assess_patient(patient: Patient) -> bool:
    bp, hr, temp = patient.blood_pressure, patient.heart_rate, patient.temperature
    risk_score = calculate_risk_score(bp, hr)

    if risk_score > 1.5:
        if hr > 100:
            return is_critical_temperature(temp)
    return False


assess_patient(Patient(blood_pressure=130.0, heart_rate=110, temperature=38.0))

True

In [18]:
print(assess_patient.trace.pretty_print())

if (((blood_pressure / 100.0000) * (heart_rate / 80.0000)) > 1.5000) (=True):
  if (heart_rate > 100) (=True):
    return (temperature > 37.5000)
